# 2. Pre-procesado de los datos - Detección de noticias falsas

José Luis Aguilera Luzania

## Introducción

**¿Qué es la detección de *Fake News*?**
La detección de noticias falsas (Fake News) es la tarea de evaluar la veracidad de las afirmaciones en las noticias. Este es un problema crítico en el Procesamiento del Lenguaje Natural (PLN) porque tanto en medios de noticias tradicionales como en medios digitales las Fake News generan un gran impacto social y político en cada individuo. Por ejemplo, la exposición a las Fake News puede generar actitudes de ineficacia, alienación y cinismo hacia ciertos candidatos políticos (Balmas, 2014).

**Objetivo de la libreta**
El objetivo de esta libreta es separar los datos que necesitamos de todo el conjunto de datos, limpiar los datos necesarios y estructurarlos para un análisis exploratorio.

## Preprocesamiento de los datos

**Librerías**

- Manipulación de datos:
    - `pandas`: Librería para manipular los datos de forma tabular.
    - `unicode`: Librería para eliminar acentos de las palabras.
    - `re`: Librería para utilizar expresiones regulares.

In [1]:
import pandas as pd
import unidecode
import re

### Leer los datos

Los datos están divididos en los archivos `train.xlsx` y `development.xlsx`, con 80% para entrenamiento y 20% para pruebas respectivamente.

Para cargar los conjuntos de datos se utilizará la librería `pandas` y su estructura `DataFrame`.

In [2]:
df_entrenamiento = pd.read_excel('Datos/train.xlsx')
df_pruebas = pd.read_excel('Datos/development.xlsx')

**Conjunto de entrenamiento**

In [3]:
print(f'Noticias, Columnas: {df_entrenamiento.shape}')
print(f'Columnas: {list(df_entrenamiento.columns)}')
print('Noticias verdaderas: {}'.format(df_entrenamiento.groupby('Category').size()[0]))
print('Noticias falsas: {}'    .format(df_entrenamiento.groupby('Category').size()[1]))

Noticias, Columnas: (676, 7)
Columnas: ['Id', 'Category', 'Topic', 'Source', 'Headline', 'Text', 'Link']
Noticias verdaderas: 338
Noticias falsas: 338


**Conjunto de pruebas**

In [4]:
print(f'Noticias, Columnas: {df_pruebas.shape}')
print(f'Columnas: {list(df_pruebas.columns)}')
print('Noticias verdaderas: {}'.format(df_pruebas.groupby('Category').size()[0]))
print('Noticias falsas: {}'    .format(df_pruebas.groupby('Category').size()[1]))

Noticias, Columnas: (295, 7)
Columnas: ['Id', 'Category', 'Topic', 'Source', 'Headline', 'Text', 'Link']
Noticias verdaderas: 142
Noticias falsas: 153


### ¿Por qué pre-procesar los datos?

Durante el análisis de los datos en la libreta anterior <<1. Análisis de los datos>> se observó que varias fuentes se encontraban escritas de forma diferente, esto provoca un incremento en el número de fuentes diferentes que se obtienen y, por lo tanto, el número de fuentes obtenidas era mucho mayor al número de fuentes real, para solucionar este problema es necesario pre-procesar los datos.

Con el propósito de evitar problemas futuros como el conteo incorrecto de las fuentes, se realizara una eliminación de datos innecesarios y una limpieza de los datos a utilizar.

**¿Cuáles son los pasos a seguir?**

1. Eliminar las columnas `Id` y `Link` de los conjuntos de datos.
2. Procesar las columnas `Category`, `Topic`, `Source`, `Headline`, `Text` como se procesaron las fuentes en la libreta anterior.

#### Eliminar las columnas `Id` y `Link`

Los datos de la columna:
- `Id` son útiles para identificar cada una de las instancias de las noticias.
- `Link` son útiles para leer el artículo original o comprobar si la noticia existe.
en este caso, no son datos necesarios o útiles para el siguiente análisis.

In [5]:
# Actualizar los dataframe.
df_entrenamiento = df_entrenamiento.drop(columns=['Id', 'Link'])
df_pruebas = df_pruebas.drop(columns=['Id', 'Link'])

In [6]:
# Comprobar las columnas.
print(f'Columnas entrenamiento: {list(df_entrenamiento.columns)}')
print(f'Columnas pruebas: {list(df_pruebas.columns)}')

Columnas entrenamiento: ['Category', 'Topic', 'Source', 'Headline', 'Text']
Columnas pruebas: ['Category', 'Topic', 'Source', 'Headline', 'Text']


#### Procesamiento de las columnas: `Category`, `Topic`, `Source`, `Headline`, `Text`

Con el fin de agilizar el proceso de procesamiento, se definirán dos funciones: `procesar_texto` y `procesar_columnas`.

- La función `procesar_textos` es la función `procesar_fuentes` de la libreta anterior, solo que renombrada para que se ajuste a su nuevo propósito.
- La función `procesar_columnas` aplicará la función `procesar_textos` a las columnas especificadas de un `DataFrame`.

Como recordatorio la función `procesar_textos` realiza lo siguiente:
1. Convertir el texto en minúsculas.
2. Eliminar acentos.
3. Eliminar todo lo que no sea una palabra.

In [7]:
def procesar_texto(texto):
    """
    Input:
        texto: una cadena de texto.

    Output:
        texto: una cadena de texto formada solo por las palabras del texto original, sin acentos, caracteres especiales o espacios extra.
    """
    # 1. Convertir el texto en minúsculas.
    texto = texto.lower()
    
    # 2. Eliminar acentos.
    texto = unidecode.unidecode(texto)
    
    # 3. Quedarnos solo con las palabras.
    texto = re.findall(r'\w+', texto)
    texto = ' '.join(texto)
    
    return texto

In [8]:
def procesar_columnas(df, columnas, funcion_procesar):
    
    for col in columnas:
        df[col] = df[col].apply(funcion_procesar)
    
    return df

In [9]:
# Lista de columnas a procesar.
columnas_a_procesar = ['Category', 'Topic', 'Source', 'Headline', 'Text']

# Procesamiento.
df_entrenamiento = procesar_columnas(df_entrenamiento, columnas_a_procesar, procesar_texto)
df_pruebas = procesar_columnas(df_pruebas, columnas_a_procesar, procesar_texto)

### Renombrar las columnas de los DataFrame.

Las columnas se encuentran escritas en inglés, pero los datos como el texto en español, para evitar inconsistencias en el análisis exploratorio las columnas se traducirán al español.

In [10]:
columnas_dict = {
    'Category': 'Categoria',
    'Topic': 'Tema',
    'Source': 'Fuente',
    'Headline': 'Encabezado',
    'Text': 'Texto'
}

df_entrenamiento.rename(columns=columnas_dict, inplace=True)
df_pruebas.rename(columns=columnas_dict, inplace=True)

### Renombrar los temas
Los temas también deberían estar en español.

Como recordatorio los temas abarcados en las noticias son los siguientes:
1. Ciencia.
2. Deportes.
3. Economía.
4. Entretenimiento.
5. Educación.
6. Política.
7. Salud.
8. Seguridad.
9. Sociedad

In [11]:
def renombrar_temas(df):
    df.loc[df['Tema'] == 'economy'      , 'Tema'] = 'economía'
    df.loc[df['Tema'] == 'education'    , 'Tema'] = 'educación'
    df.loc[df['Tema'] == 'entertainment', 'Tema'] = 'entretenimiento'
    df.loc[df['Tema'] == 'health'       , 'Tema'] = 'salud'
    df.loc[df['Tema'] == 'politics'     , 'Tema'] = 'política'
    df.loc[df['Tema'] == 'science'      , 'Tema'] = 'ciencia'
    df.loc[df['Tema'] == 'security'     , 'Tema'] = 'seguridad'
    df.loc[df['Tema'] == 'society'      , 'Tema'] = 'sociedad'
    df.loc[df['Tema'] == 'sport'        , 'Tema'] = 'deportes'

renombrar_temas(df_entrenamiento)
renombrar_temas(df_pruebas)

### Renombrar las categorías
Las categorías también serán traducidas al español.

In [12]:
def renombrar_categorias(df):
    df.loc[df['Categoria'] == 'fake'      , 'Categoria'] = 'falsa'
    df.loc[df['Categoria'] == 'true'      , 'Categoria'] = 'verdadera'

renombrar_categorias(df_entrenamiento)
renombrar_categorias(df_pruebas)

### Guardar los DataFrame en archivos de texto.

Ahora que los conjuntos se encuentran procesados y totalmente en español, para poder ser utilizados en otra libreta es necesario guardar los datos en archivos, se guardarán en archivos de extensión `.csv` llamados `datos_entrenamiento.csv` y `datos_pruebas.csv`

In [13]:
df_entrenamiento.to_csv('Datos/datos_entrenamiento.csv', index=False)
df_pruebas.to_csv('Datos/datos_pruebas.csv', index=False)